In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from lxml import html
import requests
import re
import csv
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexingTables = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)
from sklearn.model_selection import train_test_split

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english')) 

In [3]:
def removeStopwords(example_sent):
    
    word_tokens = word_tokenize(example_sent) 
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    formattedText = ""

    for word in filtered_sentence:
        
        if (len(word)>2):
            
            formattedText = formattedText + " " +word
    
    
    formattedText = formattedText.lstrip()
    
    formattedText = formattedText.rstrip()
        
    return formattedText.lower()

In [ ]:
articles = pd.read_csv('../dataset/formatted_data_articles_notnull', delimiter=',', header=None)
dataArticles = articles.iloc[:,:].values

In [ ]:
distinctArticlesIDs = []
distinctArticles = []

for articles in tqdm(dataArticles):
    
    articlePgID = articles[0]
    
    if articlePgID not in distinctArticlesIDs:
        
        distinctArticlesIDs.append(articlePgID)
        
        distinctArticles.append(articles)

In [ ]:
len(distinctArticles)

In [ ]:
with open('distinct_formatted_articles_notnull', 'w') as myfile:
        
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

    for articles in tqdm(distinctArticles):
        
        wr.writerow(articles)

In [ ]:
#distinctArticles = pd.read_csv('../dataset/distinct_formatted_articles_notnull', delimiter=',', header=None)
#dataDistinctArticles = distinctArticles.iloc[:,:].values

distinctArticles = pd.read_csv('../dataset/formatted_data_articles_notnull', delimiter=',', header=None)
dataDistinctArticles = distinctArticles.iloc[:,:].values

In [ ]:
len(distinctArticles)

## Shuffle the data in train, validation and test

In [ ]:
x_train, test_dataset = train_test_split(dataDistinctArticles, test_size=0.10, random_state=42)

## creating the test dataset

In [ ]:
with open('test_dataset_all_articles', 'w') as myfile:
          
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

    for article in tqdm(test_dataset):
        
        wr.writerow(article)

## creating the train dataset

In [ ]:
with open('train_dataset_all_articles', 'w') as myfile:
          
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

    for article in tqdm(x_train):
        
        wr.writerow(article)

In [ ]:
#comeca aqui

In [15]:
read = pd.read_csv('../dataset/train_dataset_all_articles', delimiter=',', header=None)
train_data_articles = read.iloc[:,:].values

In [16]:
train_dataset, validation_dataset = train_test_split(train_data_articles, test_size=0.05, random_state=42)

In [17]:
len(train_data_articles)

184619

In [18]:
def searchIndexingNoMatch(articleTitle):
    
    result= indexingTables.search(
        index="tables", 
        body = {
        "_source": ["tablePgID","tablePgTitle","tableSectionTitle","tableCaption","tableHeader","tableBody"],
        "from" : 0,
        "size" : 1000,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    articleTitle, 
              "fields": ["tablePgTitle"] 
            }
        }
    })
    
    return result

In [19]:
def search_index_random():
    
    result= indexingTables.search(
        index="tables", 
        body = {
        "_source": ["tablePgID","tablePgTitle","tableSectionTitle","tableCaption","tableHeader","tableBody"],
        "from" : 0,
        "size" : 100,
        "query": {
            "function_score": {
                "random_score": {}
            }
        }
    })
    
    return result

## creating the validation dataset

In [8]:
validation_nomatch, validation_match = train_test_split(validation_dataset, test_size=0.16, random_state=42)

In [9]:
len(validation_match)

1477

In [ ]:
# final_validation_dataset = []

# for article in tqdm(validation_dataset):
    
#     articleID = article[0]
#     articleTitle = article[1]
#     articleText = article[2]
#     meta_description = article[3]
#     keywords = article[5]
    
#     catch = articleTitle+" "+meta_description+" "+keywords
    
#     #getting the table title match
#     contMatch = 0
#     contNonMatch = 0
#     result = searchIndexingNoMatch(articleTitle)
#     max_score = result['hits']['max_score']
    
    
#     for hit in result['hits']['hits']:
    
#         tableID = hit['_source']['tablePgID']
#         tableTitle = (hit['_source']['tablePgTitle'])
#         tableSectionTitle = (hit['_source']['tableSectionTitle'])
#         tableCaption = (hit['_source']['tableCaption'])
#         tableHeader = (hit['_source']['tableHeader'])
#         tableBody = (hit['_source']['tableBody'])
#         table_score = hit['_score']
        
#         normalized_score = table_score/max_score
        
#         #getting match data
#         if (articleID == tableID) and (normalized_score >= 0.7): 
            
#             if contMatch < 1:
            
#                 tableIDMatch = tableID
#                 tableTitleMatch = tableTitle
#                 tableSectionTitleMatch = tableSectionTitle
#                 tableCaptionMatch = tableCaption
#                 tableHeaderMatch = tableHeader
#                 tableBodyMatch = tableBody
    
#                 final_validation_dataset.append([articleID,articleTitle, meta_description, keywords, tableIDMatch, tableTitleMatch,1])
                
#                 contMatch = contMatch + 1
                
#             else:
                
#                 contMatch = 0
#                 break
                
        
#         #getting the table title non-match
#         if (articleID != tableID) and (normalized_score < 0.3): 
            
#             if contNonMatch < 10:
                
#                 tableIDNoMatch = tableID
#                 tableTitleNoMatch = tableTitle
#                 tableSectionTitleNoMatch = tableSectionTitle
#                 tableCaptionNoMatch = tableCaption
#                 tableHeaderNoMatch = tableHeader
#                 tableBodyNoMatch = tableBody

#                 final_validation_dataset.append([articleID,articleTitle, meta_description, keywords, tableIDNoMatch,tableTitleNoMatch,0])
                
#                 contNonMatch = contNonMatch + 1
#             else:
                
#                 contNonMatch = 0
#                 break
                        

In [ ]:
final_validation_dataset = []

for article in tqdm(train_data_articles):
    
    articleID = article[0]
    articleTitle = article[1]
    articleText = article[2]
    meta_description = article[3]
    keywords = article[5]
    
    catch = articleTitle+" "+meta_description+" "+keywords
    
    #getting the table title match
    contMatch = 0
    contNonMatch = 0
    result = searchIndexingNoMatch(articleTitle)
    max_score = result['hits']['max_score']
    
    
    for hit in result['hits']['hits']:
    
        tableID = hit['_source']['tablePgID']
        tableTitle = (hit['_source']['tablePgTitle'])
        tableSectionTitle = (hit['_source']['tableSectionTitle'])
        tableCaption = (hit['_source']['tableCaption'])
        tableHeader = (hit['_source']['tableHeader'])
        tableBody = (hit['_source']['tableBody'])
        table_score = hit['_score']
        
        normalized_score = table_score/max_score
        
        #getting match data
        if (articleID == tableID) and (normalized_score >= 0.7): 
            
            if contMatch < 1:
            
                tableIDMatch = tableID
                tableTitleMatch = tableTitle
                tableSectionTitleMatch = tableSectionTitle
                tableCaptionMatch = tableCaption
                tableHeaderMatch = tableHeader
                tableBodyMatch = tableBody
    
                final_validation_dataset.append([articleID,articleTitle, meta_description, keywords, tableIDMatch, tableTitleMatch,1])
                
                contMatch = contMatch + 1
                
            else:
                
                contMatch = 0
                break

 73%|███████▎  | 134635/184619 [27:00<10:45, 77.42it/s] 

In [9]:
len(final_validation_dataset)

66647

In [ ]:
for article in tqdm(train_data_articles[0:66647]):
    
    articleID = article[0]
    articleTitle = article[1]
    articleText = article[2]
    meta_description = article[3]
    keywords = article[5]
    
    catch = articleTitle+" "+meta_description+" "+keywords
    
    #getting the table title match
    contMatch = 0
    contNonMatch = 0
    result = search_index_random()
    max_score = result['hits']['max_score']
    
    
    for hit in result['hits']['hits']:
    
        tableID = hit['_source']['tablePgID']
        tableTitle = (hit['_source']['tablePgTitle'])
        tableSectionTitle = (hit['_source']['tableSectionTitle'])
        tableCaption = (hit['_source']['tableCaption'])
        tableHeader = (hit['_source']['tableHeader'])
        tableBody = (hit['_source']['tableBody'])
        table_score = hit['_score']
        
        normalized_score = table_score/max_score
        
        #getting the table title non-match
        if (articleID != tableID) : 
            
            if contNonMatch < 5:
                
                tableIDNoMatch = tableID
                tableTitleNoMatch = tableTitle
                tableSectionTitleNoMatch = tableSectionTitle
                tableCaptionNoMatch = tableCaption
                tableHeaderNoMatch = tableHeader
                tableBodyNoMatch = tableBody

                final_validation_dataset.append([articleID,articleTitle, meta_description, keywords, tableIDNoMatch,tableTitleNoMatch,0])
                
                contNonMatch = contNonMatch + 1
            else:
                
                contNonMatch = 0
                break
                        

In [ ]:
len_validation  = len(final_validation_dataset)

In [ ]:
len_validation

In [ ]:
with open('train_dataset_random_1_5', 'w') as myfile:
          
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

    for i in tqdm(range(0,len_validation)):
    
        articlePgID = final_validation_dataset[i][0]
        articleTitle = str(final_validation_dataset[i][1])
        article_meta_description = str(final_validation_dataset[i][2])
        article_keywords = str(final_validation_dataset[i][3])
        tableID = final_validation_dataset[i][4]
        tableTitle = str(final_validation_dataset[i][5])
        match = final_validation_dataset[i][6]
        
        row = [articlePgID, articleTitle, article_meta_description, article_keywords, tableID, tableTitle, match]
        
        wr.writerow(row)

## creating the train dataset

In [21]:
train_dataset_nomatch, train_dataset_match = train_test_split(train_dataset, test_size=0.16, random_state=42)

In [ ]:
# final_train_dataset = []

# for article in tqdm(train_dataset):
    
#     articleID = article[0]
#     articleTitle = article[1]
#     articleText = article[2]
#     meta_description = article[3]
#     keywords = article[5]
    
#     catch = articleTitle+" "+meta_description+" "+keywords
    
#     #getting the table title match
#     contMatch = 0
#     contNonMatch = 0
#     result = searchIndexingNoMatch(articleTitle)
#     max_score = result['hits']['max_score']
    
#     for hit in result['hits']['hits']:
    
#         tableID = hit['_source']['tablePgID']
#         tableTitle = (hit['_source']['tablePgTitle'])
#         tableSectionTitle = (hit['_source']['tableSectionTitle'])
#         tableCaption = (hit['_source']['tableCaption'])
#         tableHeader = (hit['_source']['tableHeader'])
#         tableBody = (hit['_source']['tableBody'])
#         table_score = hit['_score']
        
#         normalized_score = table_score/max_score
        
#         if (articleID == tableID) and (normalized_score >= 0.7): 
            
#             if contMatch < 1:
            
#                 tableIDMatch = tableID
#                 tableTitleMatch = tableTitle
#                 tableSectionTitleMatch = tableSectionTitle
#                 tableCaptionMatch = tableCaption
#                 tableHeaderMatch = tableHeader
#                 tableBodyMatch = tableBody
    
#                 final_train_dataset.append([articleID,articleTitle, meta_description, keywords, tableIDMatch, tableTitleMatch,1])
                
#                 contMatch = contMatch + 1
                
#             else:
                
#                 contMatch = 0
#                 break
        
#         if (articleID != tableID) and (normalized_score < 0.3): 
            
#             if contNonMatch < 10:
                
#                 tableIDNoMatch = tableID
#                 tableTitleNoMatch = tableTitle
#                 tableSectionTitleNoMatch = tableSectionTitle
#                 tableCaptionNoMatch = tableCaption
#                 tableHeaderNoMatch = tableHeader
#                 tableBodyNoMatch = tableBody

#                 final_train_dataset.append([articleID,articleTitle, meta_description, keywords,  tableIDNoMatch,tableTitleNoMatch,0])
                
#                 contNonMatch = contNonMatch + 1
#             else:
                
#                 contNonMatch = 0
#                 break
               

In [22]:
final_train_dataset = []

for article in tqdm(train_dataset_match):
    
    articleID = article[0]
    articleTitle = article[1]
    articleText = article[2]
    meta_description = article[3]
    keywords = article[5]
    
    catch = articleTitle+" "+meta_description+" "+keywords
    
    #getting the table title match
    contMatch = 0
    contNonMatch = 0
    result = searchIndexingNoMatch(articleTitle)
    max_score = result['hits']['max_score']
    
    for hit in result['hits']['hits']:
    
        tableID = hit['_source']['tablePgID']
        tableTitle = (hit['_source']['tablePgTitle'])
        tableSectionTitle = (hit['_source']['tableSectionTitle'])
        tableCaption = (hit['_source']['tableCaption'])
        tableHeader = (hit['_source']['tableHeader'])
        tableBody = (hit['_source']['tableBody'])
        table_score = hit['_score']
        
        normalized_score = table_score/max_score
        
        if (articleID == tableID) and (normalized_score >= 0.7): 
            
            if contMatch < 1:
            
                tableIDMatch = tableID
                tableTitleMatch = tableTitle
                tableSectionTitleMatch = tableSectionTitle
                tableCaptionMatch = tableCaption
                tableHeaderMatch = tableHeader
                tableBodyMatch = tableBody
    
                final_train_dataset.append([articleID,articleTitle, meta_description, keywords, tableIDMatch, tableTitleMatch,1])
                
                contMatch = contMatch + 1
                
            else:
                
                contMatch = 0
                break
        

100%|██████████| 28063/28063 [25:32<00:00, 18.32it/s]


In [23]:
len(final_train_dataset)

10126

In [24]:
for article in tqdm(train_dataset_nomatch[0:50630]):
    
    articleID = article[0]
    articleTitle = article[1]
    articleText = article[2]
    meta_description = article[3]
    keywords = article[5]
    
    catch = articleTitle+" "+meta_description+" "+keywords
    
    #getting the table title match
    contMatch = 0
    contNonMatch = 0
    result = search_index_random()
    max_score = result['hits']['max_score']
    
    for hit in result['hits']['hits']:
    
        tableID = hit['_source']['tablePgID']
        tableTitle = (hit['_source']['tablePgTitle'])
        tableSectionTitle = (hit['_source']['tableSectionTitle'])
        tableCaption = (hit['_source']['tableCaption'])
        tableHeader = (hit['_source']['tableHeader'])
        tableBody = (hit['_source']['tableBody'])
        table_score = hit['_score']
        
        normalized_score = table_score/max_score 
        
        if (articleID != tableID): 
            
            if contNonMatch < 1:
                
                tableIDNoMatch = tableID
                tableTitleNoMatch = tableTitle
                tableSectionTitleNoMatch = tableSectionTitle
                tableCaptionNoMatch = tableCaption
                tableHeaderNoMatch = tableHeader
                tableBodyNoMatch = tableBody

                final_train_dataset.append([articleID,articleTitle, meta_description, keywords,  tableIDNoMatch,tableTitleNoMatch,0])
                
                contNonMatch = contNonMatch + 1
            else:
                
                contNonMatch = 0
                break
               

100%|██████████| 50630/50630 [12:01<00:00, 70.16it/s] 


In [25]:
len_train = len(final_train_dataset)

In [26]:
len_train

60756

In [27]:
with open('train_dataset_random_1_5', 'w') as myfile:
          
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

    for i in tqdm(range(0,len_train)):
    
        articlePgID = final_train_dataset[i][0]
        articleTitle = str(final_train_dataset[i][1])
        article_meta_description = str(final_train_dataset[i][2])
        article_keywords = str(final_train_dataset[i][3])
        tableID = final_train_dataset[i][4]
        tableTitle = str(final_train_dataset[i][5])
        match = final_train_dataset[i][6]
        
        row = [articlePgID, articleTitle, article_meta_description, article_keywords, tableID, tableTitle, match]
        
        wr.writerow(row)

100%|██████████| 60756/60756 [00:01<00:00, 56800.23it/s]
